In [1]:
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = pd.DataFrame(teams.get_teams())
#print('Number of teams fetched: {}'.format(len(nba_teams)))
#nba_teams

#create a list of all ids
team_ids = nba_teams['id'].tolist()
print(team_ids)

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [47]:
from nba_api.stats.endpoints import leaguegamefinder
NBA_list = []
#get all games for team_ids
for i in team_ids:
    games = leaguegamefinder.LeagueGameFinder(team_id_nullable = i).get_data_frames()[0]
    NBA_list.append(games)


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
for addresses in add:
    for address in addresses:
        all_address.append(address)

In [44]:
#convert NBA_list to a dataframe
NBA_Games = []

for x in NBA_list:
    for y in x:
        NBA_Games.append(y)

NameError: name 'NBA_list' is not defined

In [18]:

# Subset Season by year
# Subset the games to when the last 4 digits of SEASON_ID is in the list s.
s =['2017','2018','2019','2020','2021']
season_2017to2021 = games[games.SEASON_ID.str[-4:].isin(s)]  # subset to 2017-2021
#season_2017to2021

In [6]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Mavs, which contains their team ID
Mavs = [team for team in nba_teams if team['abbreviation'] == 'DAL'][0]
Mavs_id = Mavs['id']

In [15]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Mavs were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=Mavs_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612742,DAL,Dallas Mavericks,0022101219,2022-04-10,DAL vs. SAS,W,240,130,46,85,0.541,20,36,0.556,18,23,0.783,10,33,43,34,5,3,16,16,10.0
1,22021,1610612742,DAL,Dallas Mavericks,0022101209,2022-04-08,DAL vs. POR,W,241,128,42,79,0.532,20,45,0.444,24,31,0.774,9,42,51,25,9,5,14,19,50.0
2,22021,1610612742,DAL,Dallas Mavericks,0022101190,2022-04-06,DAL @ DET,W,238,131,45,80,0.563,15,33,0.455,26,34,0.765,8,33,41,28,8,1,10,24,18.0
3,22021,1610612742,DAL,Dallas Mavericks,0022101167,2022-04-03,DAL @ MIL,W,239,118,39,76,0.513,16,38,0.421,24,27,0.889,2,36,38,28,7,3,12,20,6.0
4,22021,1610612742,DAL,Dallas Mavericks,0022101152,2022-04-01,DAL @ WAS,L,239,103,34,78,0.436,11,38,0.289,24,26,0.923,5,31,36,14,3,3,13,16,-32.0


In [8]:
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()

In [9]:
games

[      SEASON_ID     TEAM_ID TEAM_ABBREVIATION                 TEAM_NAME  \
 0         22021  1610612737               ATL             Atlanta Hawks   
 1         12022         220               PCG             Pacers Gaming   
 2         22021  1610612748               MIA                Miami Heat   
 3         12022         201               BCG              Bucks Gaming   
 4         12022         229               WGS     Warriors Gaming Squad   
 ...         ...         ...               ...                       ...   
 29995     22015  1610612748               MIA                Miami Heat   
 29996     22015  1610612759               SAS         San Antonio Spurs   
 29997     22015  1612709911               IWA               Iowa Energy   
 29998     22015  1612709908               RGV  Rio Grande Valley Vipers   
 29999     22015  1610612764               WAS        Washington Wizards   
 
           GAME_ID   GAME_DATE      MATCHUP    WL  MIN  PTS  FGM  FGA  FG_PCT  \
 0   